In [1]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
!pip install imbalanced-learn
from imblearn.combine import SMOTEENN
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import joblib

In [2]:
df = pd.read_csv('tel_churn.csv')

In [3]:
df.head()

Unnamed: 0  SeniorCitizen  MonthlyCharges  TotalCharges  Churn  \
0           0              0           29.85         29.85      0   
1           1              0           56.95       1889.50      0   
2           2              0           53.85        108.15      1   
3           3              0           42.30       1840.75      0   
4           4              0           70.70        151.65      1   

   gender_Female  gender_Male  Partner_No  Partner_Yes  Dependents_No  ...  \
0           True        False       False         True           True  ...   
1          False         True        True        False           True  ...   
2          False         True        True        False           True  ...   
3          False         True        True        False           True  ...   
4           True        False        True        False           True  ...   

   PaymentMethod_Bank transfer (automatic)  \
0                                    False   
1                                    False   
2                                    False   
3                                     True   
4                                    False   

   PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
0                                  False                            True   
1                                  False                           False   
2                                  False                           False   
3                                  False                           False   
4                                  False                            True   

   PaymentMethod_Mailed check  tenure_group_1 - 12  tenure_group_13 - 24  \
0                       False                 True                 False   
1                        True                False                 False   
2                        True                 True                 False   
3                       False                False                 False   
4                       False                 True                 False   

   tenure_group_25 - 36  tenure_group_37 - 48  tenure_group_49 - 60  \
0                 False                 False                 False   
1                  True                 False                 False   
2                 False                 False                 False   
3                 False                  True                 False   
4                 False                 False                 False   

   tenure_group_61 - 72  
0                 False  
1                 False  
2                 False  
3                 False  
4                 False  

[5 rows x 52 columns]

In [4]:
df = df.drop('Unnamed: 0', axis=1)
df.head()

SeniorCitizen  MonthlyCharges  TotalCharges  Churn  gender_Female  \
0              0           29.85         29.85      0           True   
1              0           56.95       1889.50      0          False   
2              0           53.85        108.15      1          False   
3              0           42.30       1840.75      0          False   
4              0           70.70        151.65      1           True   

   gender_Male  Partner_No  Partner_Yes  Dependents_No  Dependents_Yes  ...  \
0        False       False         True           True           False  ...   
1         True        True        False           True           False  ...   
2         True        True        False           True           False  ...   
3         True        True        False           True           False  ...   
4        False        True        False           True           False  ...   

   PaymentMethod_Bank transfer (automatic)  \
0                                    False   
1                                    False   
2                                    False   
3                                     True   
4                                    False   

   PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
0                                  False                            True   
1                                  False                           False   
2                                  False                           False   
3                                  False                           False   
4                                  False                            True   

   PaymentMethod_Mailed check  tenure_group_1 - 12  tenure_group_13 - 24  \
0                       False                 True                 False   
1                        True                False                 False   
2                        True                 True                 False   
3                       False                False                 False   
4                       False                 True                 False   

   tenure_group_25 - 36  tenure_group_37 - 48  tenure_group_49 - 60  \
0                 False                 False                 False   
1                  True                 False                 False   
2                 False                 False                 False   
3                 False                  True                 False   
4                 False                 False                 False   

   tenure_group_61 - 72  
0                 False  
1                 False  
2                 False  
3                 False  
4                 False  

[5 rows x 51 columns]

In [5]:
x = df.drop('Churn', axis = 1)
x

SeniorCitizen  MonthlyCharges  TotalCharges  gender_Female  gender_Male  \
0                 0           29.85         29.85           True        False   
1                 0           56.95       1889.50          False         True   
2                 0           53.85        108.15          False         True   
3                 0           42.30       1840.75          False         True   
4                 0           70.70        151.65           True        False   
...             ...             ...           ...            ...          ...   
7027              0           84.80       1990.50          False         True   
7028              0          103.20       7362.90           True        False   
7029              0           29.60        346.45           True        False   
7030              1           74.40        306.60          False         True   
7031              0          105.65       6844.50          False         True   

      Partner_No  Partner_Yes  Dependents_No  Dependents_Yes  PhoneService_No  \
0          False         True           True           False             True   
1           True        False           True           False            False   
2           True        False           True           False            False   
3           True        False           True           False             True   
4           True        False           True           False            False   
...          ...          ...            ...             ...              ...   
7027       False         True          False            True            False   
7028       False         True          False            True            False   
7029       False         True          False            True             True   
7030       False         True           True           False            False   
7031        True        False           True           False            False   

      ...  PaymentMethod_Bank transfer (automatic)  \
0     ...                                    False   
1     ...                                    False   
2     ...                                    False   
3     ...                                     True   
4     ...                                    False   
...   ...                                      ...   
7027  ...                                    False   
7028  ...                                    False   
7029  ...                                    False   
7030  ...                                    False   
7031  ...                                     True   

      PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
0                                     False                            True   
1                                     False                           False   
2                                     False                           False   
3                                     False                           False   
4                                     False                            True   
...                                     ...                             ...   
7027                                  False                           False   
7028                                   True                           False   
7029                                  False                            True   
7030                                  False                           False   
7031                                  False                           False   

      PaymentMethod_Mailed check  tenure_group_1 - 12  tenure_group_13 - 24  \
0                          False                 True                 False   
1                           True                False                 False   
2                           True                 True                 False   
3                          False                False                 False   
4                          False                 True              

In [6]:
y = df['Churn']
y

0       0
1       0
2       1
3       0
4       1
       ..
7027    0
7028    0
7029    0
7030    1
7031    0
Name: Churn, Length: 7032, dtype: int64

In [7]:
x_train, x_text, y_train, y_test = train_test_split(x,y,test_size=0.2)

## Decision Tree Classifier

In [8]:
model_dt = DecisionTreeClassifier(criterion = 'gini', random_state=100, max_depth = 6, min_samples_leaf=8)

In [9]:
model_dt.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [10]:
y_pred = model_dt.predict(x_text)
y_pred

array([1, 0, 0, ..., 0, 1, 0])

In [11]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.85      0.87      0.86      1058
           1       0.58      0.53      0.56       349

    accuracy                           0.79      1407
   macro avg       0.72      0.70      0.71      1407
weighted avg       0.78      0.79      0.79      1407



In [12]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(x,y)

In [13]:
xr_train, xr_text, yr_train, yr_test = train_test_split(X_resampled,y_resampled,test_size=0.2)
model_dt_smote = DecisionTreeClassifier(criterion = 'gini', random_state=100, max_depth = 6, min_samples_leaf=8)
model_dt_smote.fit(xr_train, yr_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [14]:
y_pred_smote = model_dt_smote.predict(xr_text)

In [15]:
print(classification_report(yr_test, y_pred_smote, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92       556
           1       0.92      0.93      0.93       633

    accuracy                           0.92      1189
   macro avg       0.92      0.92      0.92      1189
weighted avg       0.92      0.92      0.92      1189



In [16]:
print(confusion_matrix(yr_test, y_pred_smote))

[[508  48]
 [ 43 590]]


## Random Forest Classifier

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
model_rf = RandomForestClassifier(n_estimators=100, criterion = 'gini', random_state=100, max_depth = 6, min_samples_leaf=8)
model_rf.fit(x_train, y_train)
y_pred_rf = model_rf.predict(x_text)
print(classification_report(y_test, y_pred_rf, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.85      0.90      0.88      1058
           1       0.63      0.52      0.57       349

    accuracy                           0.81      1407
   macro avg       0.74      0.71      0.72      1407
weighted avg       0.80      0.81      0.80      1407



In [19]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(x,y)
xr_train, xr_test, yr_train, yr_test = train_test_split(X_resampled,y_resampled,test_size=0.2)
model_smote_rf = RandomForestClassifier(n_estimators=100, criterion = 'gini', random_state=100, max_depth = 6, min_samples_leaf=8)
model_smote_rf.fit(xr_train, yr_train)
y_pred_smote_rf = model_smote_rf.predict(xr_test)
print(classification_report(yr_test, y_pred_smote_rf, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93       502
           1       0.94      0.95      0.95       666

    accuracy                           0.94      1168
   macro avg       0.94      0.94      0.94      1168
weighted avg       0.94      0.94      0.94      1168



In [20]:
print(confusion_matrix(yr_test, y_pred_smote_rf))

[[462  40]
 [ 32 634]]


In [21]:
import pickle
filename = 'model.sav'
pickle.dump(model_smote_rf, open(filename, 'wb'))

In [22]:
load_model = pickle.load(open(filename, 'rb'))

In [23]:
load_model.score(xr_test, yr_test)

0.9383561643835616